# This notebook launches the script to acquire a target and take images, meant for the scriptQueue, but via a notebook.
## This is required to debug scripts

In [1]:
import sys
import asyncio
import time

import numpy as np
import logging 
import yaml

from lsst.ts import salobj
from lsst.ts.externalscripts.auxtel.latiss_acquire_and_take_sequence import LatissAcquireAndTakeSequence

from lsst.ts.idl.enums.Script import ScriptState

In [2]:
# Temporarily needed to run at summit
import os
os.environ["LSST_DDS_DOMAIN"] = 'lsatmcs'
os.environ["OSPL_URI"] = "file:///home/patrickingraham/ospl.xml"

In [3]:
stream_handler = logging.StreamHandler(sys.stdout)
# if you want logging
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [4]:
script = LatissAcquireAndTakeSequence(index=0)  # this essentially calls the init method

In [5]:
script.attcs.atptg.cmd_raDecTarget.set(azWrapStrategy=1)

True

# Emulate how the scriptQueue launches scripts

In [6]:
script.set_state(ScriptState.UNCONFIGURED)

In [7]:
# configuration = yaml.safe_dump({"object_name": 'HD 74576', 
#                                 "acq_filter" : 'BG40',
#                                 "acq_grating" : 'empty_1',
#                                 "exposure_time_sequence" : [45, 45, 45, 45, 3, 20, 3, 20], 
#                                 "filter_sequence": ['empty_1','quadnotch1','BG40', 'BG40','RG610','RG610'], 
#                                 "grating_sequence": ['ronchi90lpmm','ronchi90lpmm','ronchi90lpmm','ronchi90lpmm','empty_1','empty_1','empty_1','empty_1'],
#                                 "doPointingModel": False,
#                                 "dataPath": '/project/shared/auxTel/rerun/quickLook'})
# print(configuration)

In [8]:
configuration = yaml.safe_dump({"object_name": 'HD 107696', 
                                "acq_filter" : 'BG40',
                                "acq_grating" : 'empty_1',
                                "exposure_time_sequence" : [4, 4, 4, 4, 1, 2, 1, 2], 
                                "filter_sequence": ['empty_1','quadnotch1','BG40', 'BG40','RG610','RG610'], 
                                "grating_sequence": ['ronchi90lpmm','ronchi90lpmm','ronchi90lpmm','ronchi90lpmm','empty_1','empty_1','empty_1','empty_1'],
                                "doPointingModel": False,
                                "dataPath": '/project/shared/auxTel/rerun/quickLook',
                                "max_acq_iter": 0 })
print(configuration)

acq_filter: BG40
acq_grating: empty_1
dataPath: /project/shared/auxTel/rerun/quickLook
doPointingModel: false
exposure_time_sequence:
- 4
- 4
- 4
- 4
- 1
- 2
- 1
- 2
filter_sequence:
- empty_1
- quadnotch1
- BG40
- BG40
- RG610
- RG610
grating_sequence:
- ronchi90lpmm
- ronchi90lpmm
- ronchi90lpmm
- ronchi90lpmm
- empty_1
- empty_1
- empty_1
- empty_1
max_acq_iter: 0
object_name: HD 107696



In [9]:
#script.updateFocus = False

In [10]:
config_data = script.cmd_configure.DataType()
config_data.config = configuration
await script.do_configure(config_data)

[('empty_1', 4, 'ronchi90lpmm'), ('quadnotch1', 4, 'ronchi90lpmm'), ('BG40', 4, 'ronchi90lpmm'), ('BG40', 4, 'ronchi90lpmm'), ('RG610', 1, 'empty_1'), ('RG610', 2, 'empty_1')]
RemoteEvent(ATMCS, 0, target) falling behind; read 64 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 65 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 16 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 17 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 17 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 18 messages
RemoteEvent(ATDome, 0, doorEncoderExtremes) falling behind; read 17 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 18 messages


In [11]:
#tmp= await script.latiss.take_bias(nbias=1)

In [12]:
#script.max_acq_iter = 0  # this will not perform any offsetting - but will slew.
#script.do_acquire=True # Set this to true or it won't slew
await script.run()
print('DONE!')

Slewing to HD 107696: 12 22 49.4319 -57 40 34.066
Parallactic angle: 31.869314670627688 | Sky Angle: 61.68697402152293
Sending command
Stop tracking.
Mount tracking state is 9
RemoteEvent(ATMCS, 0, target) falling behind; read 20 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 20 messages
Tracking state: 10.
Tracking state: 8.
In Position: True.
In Position: False.
Scheduling check coroutines
process as completed...
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
atdome: <State.ENABLED: 2>
atdometrajectory: <State.ENABLED: 2>
[Telescope] delta Alt = -010.061 | delta Az = +007.836 delta N1 = -000.000 delta N2 = -038.845 [Dome] delta Az = +002.932
[Telescope] delta Alt = -007.395 | delta Az = +004.469 delta N1 = -000.000 delta N2 = -033.206 [Dome] delta Az = +002.932
[Telescope] delta Alt = -001.829 | delta Az = +000.142 delta N1 = -000.000 delta N2 = -027.220 [Dome] de

In [4]:
# doesn't appear to take the offset?
#await script.attcs.ataos.cmd_applyFocusOffset.set_start(offset=-0.07, timeout=20)

In [17]:
#await script.attcs.ataos.cmd_applyAxisOffset.set_start(axis='z', offset=-0.07, timeout=20)

In [14]:
#await script.attcs.ataos.cmd_resetOffset.start()

In [ ]:
configuration = yaml.safe_dump({"object_name": 'HD 107696', 
                                "acq_filter" : 'BG40',
                                "acq_grating" : 'empty_1',
                                "exposure_time_sequence" : [4, 4, 4, 4, 1, 2, 1, 2], 
                                "filter_sequence": ['empty_1','quadnotch1','BG40', 'BG40','RG610','RG610'], 
                                "grating_sequence": ['ronchi90lpmm','ronchi90lpmm','ronchi90lpmm','ronchi90lpmm','empty_1','empty_1','empty_1','empty_1'],
                                "doPointingModel": False,
                                "dataPath": '/project/shared/auxTel/rerun/quickLook',
                                "max_acq_iter": 0 })

In [ ]:
#await script.attcs.slew_object('CD-36  7018')

In [ ]:
target_list=["BD-14  2342", "HD  75942", "HD  85754", "HD  92944", "HD 101492"]
# Reverse the targets?
#target_list=target_list[::-1]
print(target_list)

In [ ]:
for target in target_list:
    script.set_state(ScriptState.UNCONFIGURED)
    
    configuration = yaml.safe_dump({"object_name": target, 
                                "exposure_time": 2, 
                                "filter": 'empty_1', 
                                "grating": 'empty_1',
                                "doPointingModel": True,
                                "dataPath": '/project/shared/auxTel/rerun/quickLook'})
    print(configuration)
    
    config_data = script.cmd_configure.DataType()
    config_data.config = configuration
    await script.do_configure(config_data)
    
    await script.run()

In [ ]:
print("Done")

In [17]:
await script.attcs.atmcs.cmd_stopTracking.start()

In [18]:
await script.attcs.point_azel(az=240,el=80)

Sending command
Stop tracking.
Mount tracking state is 8
In Position: False.
Scheduling check coroutines
process as completed...
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
[Telescope] delta Alt = +022.669 | delta Az= -003.841 delta N1 = +000.000 delta N2 = -000.420 
[Telescope] delta Alt = +022.483 | delta Az= -001.235 delta N1 = -000.000 delta N2 = +000.055 
[Telescope] delta Alt = +020.400 | delta Az= +000.000 delta N1 = +000.000 delta N2 = +000.424 
[Telescope] delta Alt = +017.625 | delta Az= +000.000 delta N1 = +000.000 delta N2 = +000.725 
[Telescope] delta Alt = +014.020 | delta Az= +000.000 delta N1 = +000.000 delta N2 = +000.942 
[Telescope] delta Alt = +008.103 | delta Az= +000.000 delta N1 = +000.000 delta N2 = +000.872 
[Telescope] delta Alt = +003.174 | delta Az= +000.000 delta N1 = +000.000 delta N2 = +000.707 
[Telescope] delta Alt = +001.114 | delta Az= +000.000 delta N1 = 

In [14]:
# stops the messaging output if you had to stop the above cell
await script.attcs.cancel_not_done(script.attcs.scheduled_coro)

In [15]:
await script.attcs.atptg.cmd_stopTracking.start()

In [ ]:
await script.latiss.take_object(exptime=2)

In [ ]:
await script.attcs.offset_xy(x=45.1975, y=-48.6315)

In [ ]:
await script.latiss.take_object(exptime=5, grating='empty_1', filter='BG40')

In [ ]:
await script.latiss.take_engtest(exptime=1, grating='empty_1', filter='RG610')

In [ ]:
configuration = yaml.safe_dump({"object_name": 'HD 107696', 
                                "acq_filter" : 'BG40',
                                "acq_grating" : 'empty_1',
                                "exposure_time_sequence" : [15, 15, 15, 15, 15, 15, 15, 15], 
                                "filter_sequence": ['empty_1','quadnotch1','BG40', 'BG40','RG610','RG610'], 
                                "grating_sequence": ['ronchi90lpmm','ronchi90lpmm','ronchi90lpmm','ronchi90lpmm','empty_1','empty_1','empty_1','empty_1'],
                                "doPointingModel": False,
                                "dataPath": '/project/shared/auxTel/rerun/quickLook',
                                "max_acq_iter": 0 })